### Imports

In [1]:
from importlib import reload
from itertools import chain

import pandas as pd
import numpy as np
import networkx as nx
import nx_parallel as nxp
import scipy

from os import listdir
from os.path import isfile, join

# Importieren und neu laden (damit aktuelle Änderungen übernommen werden)
from helper import graph_analyzer, text_preprocesser
reload(text_preprocesser)
reload(graph_analyzer)


from helper.text_preprocesser import prepare_text, prepare_text_with_libraries, convert_preprocessed_tokens_to_graph
from helper.graph_analyzer import parallel_get_distance_measures, parallel_get_betweenness_list

### Optionen setzen

In [2]:
PARALLEL = False
nx.config.backends.parallel.active = True
nx.config.backends.parallel.n_jobs = 4
print(nx.config)

NetworkXConfig(backend_priority=BackendPriorities(algos=[], generators=[]), backends=Config(parallel=ParallelConfig(active=True, backend='loky', n_jobs=4, verbose=0, temp_folder=None, max_nbytes='1M', mmap_mode='r', prefer=None, require=None, inner_max_num_threads=None, backend_params={})), cache_converted_graphs=True, fallback_to_nx=False, warnings_to_ignore=set())


### Dateipfade laden

In [3]:
# Allgemeiner Pfad zu den Daten
DATA_PATH = "data/input/"

# Liste mit Dateinamen von Büchern
file_name_list = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f))]

# Liste mit Inhalten von Dateien
file_content_list = [" ".join(open(join(DATA_PATH, f)).readlines()) for f in file_name_list]


### Dataframe mit Dateinamen und Inhalt befüllen

In [4]:
# Tabelle erstellen mit Spalten "title" und "content"
df = pd.DataFrame({'file_name': file_name_list, 'file_content': file_content_list})


def extract_metadata_from_file_name(file_name):
    splitted_name = file_name[:-4].split("_")
    author = splitted_name[0]
    title = splitted_name[1]
    language = splitted_name[2]
    return author, title, language

file_metadata = df["file_name"].apply(extract_metadata_from_file_name).apply(pd.Series)

df[["author", "title", "language"]] = file_metadata

print(df)

                        file_name  \
0  Hesse_Camenzind_en_cleaned.txt   
1                  ALL_ALL_DE.txt   
2  Hesse_Camenzind_de_cleaned.txt   

                                        file_content author      title  \
0  IN THE BEGINNING was the myth. God, in his sea...  Hesse  Camenzind   
1  IN THE BEGINNING was the myth. God, in his sea...    ALL        ALL   
2  Im Anfang war der Mythus. Wie der große Gott i...  Hesse  Camenzind   

  language  
0       en  
1       DE  
2       de  


### Text vorbereiten und Graphen erstellen

In [5]:
df["prepared_text"] = df["file_content"].apply(prepare_text_with_libraries)

df["graph"] = df["prepared_text"].apply(lambda g: convert_preprocessed_tokens_to_graph(g, 1))

### Metriken berechnen

In [6]:
# Basismetriken: Knotenanzahl und Kantenanzahl
df["node_count"] = df["graph"].apply(lambda g: len(g.nodes))
df["edge_count"] = df["graph"].apply(lambda g: len(g.edges))

# Knotengrade
df["degree_list"] = df["graph"].apply(lambda g: np.array([deg for node, deg in g.degree ]))
# Alternative von ChatGPT: dataframe["degree_list"] = dataframe["graph"].apply(lambda g: np.fromiter(dict(g.degree).values(), dtype=float))
df["average_degree"] = df["degree_list"].apply(lambda degree_list: np.mean(degree_list))
df["median_degree"] = df["degree_list"].apply(lambda degree_list: np.median(degree_list))

# Falls eingeschaltet: Parallele Berechnungen
if PARALLEL:
    distance_measures = df["graph"].apply(parallel_get_distance_measures).apply(pd.Series)
    df[["diameter", "average_distance"]] = distance_measures
    df["betweenness_list"] = df["graph"].apply(parallel_get_betweenness_list)

# Ansonsten (NICHT parallel)
else:
    df["diameter"] = df["graph"].apply(lambda g: nx.diameter(g))
    df["average_distance"] = df["graph"].apply(lambda g: nx.average_shortest_path_length(g))
    df["betweenness_list"] = df["graph"].apply(lambda g : np.array( list(nx.betweenness_centrality(g).values()) ))

# Parameter Lambda der Exponentialverteilung bestimmen

df["lambda"] = df["degree_list"].apply(lambda d: 1 / scipy.stats.expon.fit(d)[1])

# Betweenness aus Liste der Einzelwerte
df["betweenness_min"] = df["betweenness_list"].apply(np.min)
df["betweenness_max"] = df["betweenness_list"].apply(np.max)
df["betweenness_average"] = df["betweenness_list"].apply(np.mean)
df["betweenness_standard_deviation"] = df["betweenness_list"].apply(np.std)


# Clustering
df["average_clustering"] = df["graph"].apply(lambda g: nx.average_clustering(g))




### Speichern der Tabelle in Datei

In [ ]:
# Nur bestimmte Spalten sollen in Datei geschrieben werden
attributes_visible_in_file = [
    "author",
    "title",
    "language",
    "node_count",
    "edge_count",
    "average_degree",
    "median_degree",
    "diameter",
    "average_distance",
    "betweenness_min",
    "betweenness_max",
    "betweenness_average",
    "betweenness_standard_deviation",
    "average_clustering",
    "lambda"
]

df[attributes_visible_in_file].to_csv("data/output/output.csv", index=False)

print(df)

                        file_name  \
0  Hesse_Camenzind_en_cleaned.txt   
1                  ALL_ALL_DE.txt   
2  Hesse_Camenzind_de_cleaned.txt   

                                        file_content author      title  \
0  IN THE BEGINNING was the myth. God, in his sea...  Hesse  Camenzind   
1  IN THE BEGINNING was the myth. God, in his sea...    ALL        ALL   
2  Im Anfang war der Mythus. Wie der große Gott i...  Hesse  Camenzind   

  language                                      prepared_text  \
0       en  [THE, BEGINNING, the, myth, God, His, Search, ...   
1       DE  [THE, BEGINNING, the, myth, God, His, Search, ...   
2       de  [Im, Anfang, Mythus, groß, Gott, Seele, Inder,...   

                                               graph  node_count  edge_count  \
0  (THE, BEGINNING, the, myth, God, His, Search, ...        2022        4885   
1  (THE, BEGINNING, the, myth, God, His, Search, ...        2130        5016   
2  (Im, Anfang, Mythus, groß, Gott, Seele, Inder,... 